[View in Colaboratory](https://colab.research.google.com/github/zhangyi02/ZhangYi/blob/master/CNN_scene15_augmentation.ipynb)

In [1]:
! wget http://www-cvr.ai.uiuc.edu/ponce_grp/data/scene_categories/scene_categories.zip
import zipfile
import os
def un_zip():
    """unzip zip file"""
    zip_file = zipfile.ZipFile("scene_categories.zip")
    if os.path.isdir("scene_categories"):
        pass
    else:
        os.mkdir("scene_categories")
    for names in zip_file.namelist():
        zip_file.extract(names,"scene_categories")
    zip_file.close()
un_zip()

--2018-07-24 01:31:57--  http://www-cvr.ai.uiuc.edu/ponce_grp/data/scene_categories/scene_categories.zip
Resolving www-cvr.ai.uiuc.edu (www-cvr.ai.uiuc.edu)... 128.174.227.253
Connecting to www-cvr.ai.uiuc.edu (www-cvr.ai.uiuc.edu)|128.174.227.253|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 86098604 (82M) [application/zip]
Saving to: ‘scene_categories.zip’

scene_categories.zi 100%[===================>]  82.11M  32.7MB/s    in 2.5s    

2018-07-24 01:31:59 (32.7 MB/s) - ‘scene_categories.zip’ saved [86098604/86098604]



In [2]:
import tensorflow as tf
tf.reset_default_graph()
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

Found GPU at: /device:GPU:0


In [0]:
# Load data
import cv2
import os
import numpy as np
import random
import tensorflow as tf
import pdb
from sklearn.preprocessing import scale


def pre_processing():
    img_name_list = {}
    img_name_train_list = {}
    img_name_test_list = {}
    img_name_train_total = []
    img_name_test_total = []
    label_train_total = []
    label_test_total = []
    img_class = ['bedroom', 'CALsuburb', 'industrial', 'kitchen', 'livingroom',
                 'MITcoast', 'MITforest', 'MIThighway', 'MITinsidecity', 'MITmountain',
                 'MITopencountry', 'MITstreet', 'MITtallbuilding', 'PARoffice', 'store']
    for i in range(len(img_class)):
        path_class = 'scene_categories/' + img_class[i]
        img_name_list[i] = os.listdir(path_class)  # 列出文件夹下所有的目录与文件

        img_name_train_list[i] = random.sample(img_name_list[i], 100)
        img_name_test_list[i] = list(set(img_name_list[i]) - set(img_name_train_list[i]))
        #   至此读出了所有文件路径存储于img_train_list, img_test_list两个字典中，key为label的index，对应的为该类下所有图片

        img_name_train_total += img_name_train_list[i]
        label_train_total += [i] * len(img_name_train_list[i])
        img_name_test_total += img_name_test_list[i]
        label_test_total += [i] * len(img_name_test_list[i])
    #   将各个label训练、测试数据分别合并为一个list

    train_index = np.arange(len(img_name_train_total))
    np.random.shuffle(train_index)
    img_name_train_total_shuffle = [img_name_train_total[index] for index in train_index]
    label_train_total_shuffle = [label_train_total[index] for index in train_index]
    # shuffle 数据

    batch_size = 50
    batch_nums = len(img_name_train_total_shuffle) // batch_size
    img_name_batch = {}
    label_batch = {}
    for batch_num in range(batch_nums):
        img_name_batch[batch_num] = img_name_train_total_shuffle[batch_num * batch_size: (batch_num + 1) * batch_size]
        label_batch[batch_num] = label_train_total_shuffle[batch_num * batch_size: (batch_num + 1) * batch_size]

    return img_name_batch, label_batch, img_class, img_name_test_total, label_test_total

In [0]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, 2, 2, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    x = tf.reshape(x, shape=[-1, img_size, img_size, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Add Convolution layer _zy
    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
    # Max pooling (down-sampling)
    conv3 = maxpool2d(conv3, k=2)
    print(conv3.shape)

    # Fully connected layer
    # Reshape conv3 output to fit fully connected layer input
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)
    
    
    fc2 = tf.add(tf.matmul(fc1, weights['wd2']), biases['bd2'])
    fc2 = tf.nn.relu(fc2)
    # Apply Dropout
    fc2 = tf.nn.dropout(fc2, dropout)
    
    fc3 = tf.add(tf.matmul(fc2, weights['wd3']), biases['bd3'])
    fc3 = tf.nn.relu(fc3)
    # Apply Dropout
    fc3 = tf.nn.dropout(fc3, dropout)


    # Output, class prediction
    out = tf.add(tf.matmul(fc3, weights['out']), biases['out'])
    return out

def normlization():
    img_data_max, img_data_min = 0, 0
    for batch_num in range(len(img_name_batch)):
        img_name = img_name_batch[batch_num]
        label = label_batch[batch_num]
        for i in range(len(img_name)):
            img_full_name = 'scene_categories/' + img_class[label[i]] + '/' + img_name[i]
            img = cv2.imread(img_full_name)[:, :, 0]
            img = cv2.resize(img, (img_size+2, img_size+2))
            img_data = np.reshape(img,(-1,))
            if img_data_max < max(img_data):
                img_data_max = max(img_data)
            if img_data_min > min(img_data):
                img_data_min = min(img_data)
    return img_data_max, img_data_min


def batch():
    # read jpg
    batch_num = np.random.randint(0, len(img_name_batch))
    img_name = img_name_batch[batch_num]
    label = label_batch[batch_num]
    img_data = np.zeros((len(img_name)*9, img_size * img_size))
    label2 = np.zeros((len(img_name)*9, len(img_class)))
    for i in range(len(img_name)):
        img_full_name = 'scene_categories/' + img_class[label[i]] + '/' + img_name[i]
        img = cv2.imread(img_full_name)[:, :, 0]
        img = cv2.resize(img, (img_size+2, img_size+2))
        d = -1
        for j in range(3):
            for k in range(3):
                d += 1
                img_data_ = np.reshape(img[j:j+img_size, k:k+img_size],(-1,))
#                 img_data_min = min(img_data_)
#                 img_data_max = max(img_data_)
                img_data_nor = (img_data_ - img_data_min) / (img_data_max - img_data_min)
                img_data[i*9+d] = img_data_nor
                label2[i*9+d, label[i]] = 1
#                 img_ = scale(img[j:j+img_size, k:k+img_size])
#                 img_data[i*9+d] = np.reshape(img_,(-1,))
#                 label2[i*9+d, label[i]] = 1
    return img_data, label2


def batch_test():
    # read jpg
    img_name = img_name_test
    label = label_test
    img_data = np.zeros((len(img_name), img_size * img_size))
    label2 = np.zeros((len(img_name), len(img_class)))
    for i in range(len(img_name)):
        img_full_name = 'scene_categories/' + img_class[label[i]] + '/' + img_name[i]
        img = cv2.imread(img_full_name)[:, :, 0]
        img = cv2.resize(img, (img_size, img_size))
        img_data_ = np.reshape(img,(-1,))
#         img_data_min = min(img_data_)
#         img_data_max = max(img_data_)
        img_data_nor = (img_data_ - img_data_min) / (img_data_max - img_data_min)
        img_data[i] = img_data_nor
        label2[i, label[i]] = 1
#         img = scale(img)
#         img_data[i] = np.reshape(img, (-1,))
#         label2[i, label[i]] = 1
    return img_data, label2


img_name_batch, label_batch, img_class, img_name_test, label_test = pre_processing()
img_size = 64
img_data_max, img_data_min = normlization()

# Training Parameters
learning_rate = 0.001
num_steps = 2000
display_step = 100

# Network Parameters
num_input = img_size * img_size  # MNIST data input (img shape: 28*28)
num_classes = 15  # MNIST total classes (0-9 digits)
dropout = 0.5  # Dropout, probability to keep units

# tf Graph input
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32)  # dropout (keep probability)




# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([3, 3, 1, 64]) * np.sqrt(2/1/9)),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([3, 3, 64, 128]) * np.sqrt(2/64/9)),
    # 5X5 conv, 64 inputs, 128 outputs
    'wc3': tf.Variable(tf.random_normal([3, 3, 128, 256]) * np.sqrt(2/128/9)),
    # fully connected, 8*8*128 inputs, 1024 outputs  ## decided by maxpooling k
    'wd1': tf.Variable(tf.random_normal(
        [int(img_size/8) * int(img_size/8) * 256, 512]) * np.sqrt(2/256/64)),
    # fully connected, 1024 inputs, 512 outputs
    'wd2': tf.Variable(tf.random_normal([512, 256]) * np.sqrt(2/512)),
    'wd3': tf.Variable(tf.random_normal([256, 256]) * np.sqrt(2/256)),
    # 512 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([256, num_classes]) * np.sqrt(2/256))
}

biases = {
    'bc1': tf.Variable(tf.zeros([64])),
    'bc2': tf.Variable(tf.zeros([128])),
    'bc3': tf.Variable(tf.zeros([256])),
    'bd1': tf.Variable(tf.zeros([512])),
    'bd2': tf.Variable(tf.zeros([256])),
    'bd3': tf.Variable(tf.zeros([256])),
    'out': tf.Variable(tf.zeros([num_classes]))
}

# Construct model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)


# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training
with tf.Session(config = config) as sess:
    # Run the initializer
    print('running cnn')
    sess.run(init)
    
    for step in range(1, num_steps + 1):
        if step > 1000:
            learning_rate = 0.0005
        # print('iter: ', step)
        batch_x, batch_y = batch()
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: 0.8})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y,
                                                                 keep_prob: 1})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc) +'<br>')

    print("Optimization Finished!")

    test_x, test_y = batch_test()
    print('Testing Accuracy:', \
          sess.run(accuracy, feed_dict={X: test_x,
                                        Y: test_y,
                                        keep_prob: 1.0}))
    print('<br>learning rate = %.4f, img size = %i, dropout = %.3f' %(learning_rate, img_size, dropout))

(?, 8, 8, 256)
running cnn
Step 1, Minibatch Loss= 194.9240, Training Accuracy= 0.100<br>
Step 100, Minibatch Loss= 2.4974, Training Accuracy= 0.200<br>
Step 200, Minibatch Loss= 2.3128, Training Accuracy= 0.304<br>
Step 300, Minibatch Loss= 2.0077, Training Accuracy= 0.364<br>
Step 400, Minibatch Loss= 2.0381, Training Accuracy= 0.316<br>
Step 500, Minibatch Loss= 1.9727, Training Accuracy= 0.396<br>
Step 600, Minibatch Loss= 1.7555, Training Accuracy= 0.496<br>
Step 700, Minibatch Loss= 1.6250, Training Accuracy= 0.447<br>
Step 800, Minibatch Loss= 1.5435, Training Accuracy= 0.507<br>
Step 900, Minibatch Loss= 1.1101, Training Accuracy= 0.624<br>
Step 1000, Minibatch Loss= 0.8418, Training Accuracy= 0.680<br>
Step 1100, Minibatch Loss= 0.6153, Training Accuracy= 0.824<br>
Step 1200, Minibatch Loss= 0.2503, Training Accuracy= 0.947<br>
Step 1300, Minibatch Loss= 0.0684, Training Accuracy= 0.980<br>
Step 1400, Minibatch Loss= 0.0519, Training Accuracy= 0.980<br>
Step 1500, Minibatch Lo

w\*1 seems doesn't work<br>

Step 1, Minibatch Loss= 9.1069, Training Accuracy= 0.080<br>
Step 100, Minibatch Loss= 1.2083, Training Accuracy= 0.631<br>
Step 200, Minibatch Loss= 0.4360, Training Accuracy= 0.900<br>
Step 300, Minibatch Loss= 0.0265, Training Accuracy= 1.000<br>
Step 400, Minibatch Loss= 0.0458, Training Accuracy= 0.984<br>
Step 500, Minibatch Loss= 0.0001, Training Accuracy= 1.000<br>
Step 600, Minibatch Loss= 0.0000, Training Accuracy= 1.000<br>
Step 700, Minibatch Loss= 0.0005, Training Accuracy= 1.000<br>
Step 800, Minibatch Loss= 0.0001, Training Accuracy= 1.000<br>
Step 900, Minibatch Loss= 0.0005, Training Accuracy= 1.000<br>
Step 1000, Minibatch Loss= 0.0001, Training Accuracy= 1.000<br>
Step 1100, Minibatch Loss= 0.0000, Training Accuracy= 1.000<br>
Step 1200, Minibatch Loss= 0.0010, Training Accuracy= 1.000<br>
Step 1300, Minibatch Loss= 0.0001, Training Accuracy= 1.000<br>
Step 1400, Minibatch Loss= 0.0000, Training Accuracy= 1.000<br>
Step 1500, Minibatch Loss= 0.0000, Training Accuracy= 1.000<br>
Step 1600, Minibatch Loss= 0.0000, Training Accuracy= 1.000<br>
Step 1700, Minibatch Loss= 0.0000, Training Accuracy= 1.000<br>
Step 1800, Minibatch Loss= 0.0001, Training Accuracy= 1.000<br>
Step 1900, Minibatch Loss= 0.0052, Training Accuracy= 1.000<br>
Step 2000, Minibatch Loss= 0.0001, Training Accuracy= 1.000<br>
Optimization Finished!
Testing Accuracy: 0.5329983
<br>learning rate = 0.001->0.005(1000step), img size = 64, dropout = 0.500<br>
totally 0-1 normalization & augmentation = 9<br>
w / np.sqrt(2/input_size); b = 0


Step 1, Minibatch Loss= 50.6407, Training Accuracy= 0.140<br>
Step 100, Minibatch Loss= 1.7829, Training Accuracy= 0.491<br>
Step 200, Minibatch Loss= 0.6602, Training Accuracy= 0.811<br>
Step 300, Minibatch Loss= 0.0527, Training Accuracy= 0.991<br>
Step 400, Minibatch Loss= 0.0596, Training Accuracy= 0.978<br>
Step 500, Minibatch Loss= 0.0009, Training Accuracy= 1.000<br>
Step 600, Minibatch Loss= 0.0006, Training Accuracy= 1.000<br>
Step 700, Minibatch Loss= 0.0002, Training Accuracy= 1.000<br>
Step 800, Minibatch Loss= 0.0001, Training Accuracy= 1.000<br>
Step 900, Minibatch Loss= 0.0000, Training Accuracy= 1.000<br>
Step 1000, Minibatch Loss= 0.0000, Training Accuracy= 1.000<br>
Step 1100, Minibatch Loss= 0.0000, Training Accuracy= 1.000<br>
Step 1200, Minibatch Loss= 0.0000, Training Accuracy= 1.000<br>
Step 1300, Minibatch Loss= 0.0000, Training Accuracy= 1.000<br>
Step 1400, Minibatch Loss= 0.0000, Training Accuracy= 1.000<br>
Step 1500, Minibatch Loss= 0.0000, Training Accuracy= 1.000<br>
Step 1600, Minibatch Loss= 0.0008, Training Accuracy= 1.000<br>
Step 1700, Minibatch Loss= 0.0000, Training Accuracy= 1.000<br>
Step 1800, Minibatch Loss= 0.0000, Training Accuracy= 1.000<br>
Step 1900, Minibatch Loss= 0.0000, Training Accuracy= 1.000<br>
Step 2000, Minibatch Loss= 0.0000, Training Accuracy= 1.000<br>
Optimization Finished!
Testing Accuracy: 0.54237854
<br>learning rate = 0.001->0.005(1000 step), img size = 64, dropout = 0.500<br>
totally 0-1 normalization & augmentation = 9<br>
w / np.sqrt(1/input_size); b = 0

Step 1, Minibatch Loss= 102.5965, Training Accuracy= 0.080<br>
Step 100, Minibatch Loss= 2.5593, Training Accuracy= 0.171<br>
Step 200, Minibatch Loss= 2.2665, Training Accuracy= 0.329<br>
Step 300, Minibatch Loss= 2.0441, Training Accuracy= 0.307<br>
Step 400, Minibatch Loss= 1.6417, Training Accuracy= 0.489<br>
Step 500, Minibatch Loss= 1.5321, Training Accuracy= 0.496<br>
Step 600, Minibatch Loss= 1.5222, Training Accuracy= 0.549<br>
Step 700, Minibatch Loss= 1.1719, Training Accuracy= 0.642<br>
Step 800, Minibatch Loss= 0.9331, Training Accuracy= 0.749<br>
Step 900, Minibatch Loss= 0.3650, Training Accuracy= 0.909<br>
Step 1000, Minibatch Loss= 0.3061, Training Accuracy= 0.902<br>
Step 1100, Minibatch Loss= 0.2839, Training Accuracy= 0.922<br>
Step 1200, Minibatch Loss= 0.1120, Training Accuracy= 0.982<br>
Step 1300, Minibatch Loss= 0.0315, Training Accuracy= 1.000<br>
Step 1400, Minibatch Loss= 0.0296, Training Accuracy= 1.000<br>
Step 1500, Minibatch Loss= 0.0148, Training Accuracy= 1.000<br>
Optimization Finished!
Testing Accuracy: 0.46968174
<br>learning rate = 0.0010, img size = 64, dropout = 0.500<br>
totally 0-1 normalization & augmentation = 9<br>


Step 1, Minibatch Loss= 272.6266, Training Accuracy= 0.060<br>
Step 100, Minibatch Loss= 2.6149, Training Accuracy= 0.118<br>
Step 200, Minibatch Loss= 2.3687, Training Accuracy= 0.300<br>
Step 300, Minibatch Loss= 2.2466, Training Accuracy= 0.229<br>
Step 400, Minibatch Loss= 2.2527, Training Accuracy= 0.260<br>
Step 500, Minibatch Loss= 1.9124, Training Accuracy= 0.451<br>
Step 600, Minibatch Loss= 1.7814, Training Accuracy= 0.404<br>
Step 700, Minibatch Loss= 1.2468, Training Accuracy= 0.596<br>
Step 800, Minibatch Loss= 0.5802, Training Accuracy= 0.860<br>
Step 900, Minibatch Loss= 0.5665, Training Accuracy= 0.829<br>
Step 1000, Minibatch Loss= 0.6768, Training Accuracy= 0.831<br>
Step 1100, Minibatch Loss= 0.1627, Training Accuracy= 0.956<br>
Step 1200, Minibatch Loss= 0.0648, Training Accuracy= 0.998<br>
Step 1300, Minibatch Loss= 0.0248, Training Accuracy= 1.000<br>
Step 1400, Minibatch Loss= 0.0053, Training Accuracy= 1.000<br>
Step 1500, Minibatch Loss= 0.0018, Training Accuracy= 1.000<br>
Optimization Finished!
Testing Accuracy: 0.4639866
<br>learning rate = 0.0010, img size = 64, dropout = 0.500<br>
batch 0-1 normalization & augmentation = 9

Step 1, Minibatch Loss= 3.3431, Training Accuracy= 0.100<br>
Step 100, Minibatch Loss= 2.6898, Training Accuracy= 0.120<br>
Step 200, Minibatch Loss= 2.5636, Training Accuracy= 0.120<br>
Step 300, Minibatch Loss= 2.1833, Training Accuracy= 0.255<br>
Step 400, Minibatch Loss= 1.8982, Training Accuracy= 0.320<br>
Step 500, Minibatch Loss= 1.2663, Training Accuracy= 0.545<br>
Step 600, Minibatch Loss= 1.0513, Training Accuracy= 0.630<br>
Step 700, Minibatch Loss= 0.4957, Training Accuracy= 0.810<br>
Step 800, Minibatch Loss= 0.3374, Training Accuracy= 0.870<br>
Step 900, Minibatch Loss= 0.1788, Training Accuracy= 0.955<br>
Step 1000, Minibatch Loss= 0.1225, Training Accuracy= 0.965<br>
Step 1100, Minibatch Loss= 0.0578, Training Accuracy= 0.985<br>
Step 1200, Minibatch Loss= 0.0166, Training Accuracy= 1.000<br>
Step 1300, Minibatch Loss= 0.0099, Training Accuracy= 1.000<br>
Step 1400, Minibatch Loss= 0.0051, Training Accuracy= 1.000<br>
Step 1500, Minibatch Loss= 0.0043, Training Accuracy= 1.000<br>
Optimization Finished!
Testing Accuracy: 0.44087103
<br>learning rate = 0.0010, img size = 64, dropout = 0.500<br>
0-score normalization & augmentation = 4

Step 1, Minibatch Loss= 3.2704, Training Accuracy= 0.140<br>
Step 100, Minibatch Loss= 2.7297, Training Accuracy= 0.060<br>
Step 200, Minibatch Loss= 2.7280, Training Accuracy= 0.020<br>
Step 300, Minibatch Loss= 2.7162, Training Accuracy= 0.060<br>
Step 400, Minibatch Loss= 2.5011, Training Accuracy= 0.180<br>
Step 500, Minibatch Loss= 1.7072, Training Accuracy= 0.360<br>
Step 600, Minibatch Loss= 1.3197, Training Accuracy= 0.630<br>
Step 700, Minibatch Loss= 0.9435, Training Accuracy= 0.640<br>
Step 800, Minibatch Loss= 0.6959, Training Accuracy= 0.760<br>
Step 900, Minibatch Loss= 0.0725, Training Accuracy= 1.000<br>
Step 1000, Minibatch Loss= 0.0783, Training Accuracy= 0.980<br>
Step 1100, Minibatch Loss= 0.0936, Training Accuracy= 0.980<br>
Step 1200, Minibatch Loss= 0.0287, Training Accuracy= 1.000<br>
Step 1300, Minibatch Loss= 0.0080, Training Accuracy= 1.000<br>
Step 1400, Minibatch Loss= 0.0542, Training Accuracy= 0.980<br>
Step 1500, Minibatch Loss= 0.0072, Training Accuracy= 1.000<br>
Optimization Finished!
Testing Accuracy: 0.41038525
<br>learning rate = 0.0010, img size = 64, dropout = 0.500<br>
3\*conv:64,128,256<br>
3\*fc:512,256,256<br>
augmentation = 2

Step 1, Minibatch Loss= 3.2841, Training Accuracy= 0.100<br>
Step 100, Minibatch Loss= 2.7871, Training Accuracy= 0.060<br>
Step 200, Minibatch Loss= 2.3767, Training Accuracy= 0.200<br>
Step 300, Minibatch Loss= 2.0556, Training Accuracy= 0.300<br>
Step 400, Minibatch Loss= 1.8364, Training Accuracy= 0.300<br>
Step 500, Minibatch Loss= 1.4076, Training Accuracy= 0.500<br>
Step 600, Minibatch Loss= 1.0426, Training Accuracy= 0.640<br>
Step 700, Minibatch Loss= 0.7521, Training Accuracy= 0.740<br>
Step 800, Minibatch Loss= 0.4012, Training Accuracy= 0.880<br>
Step 900, Minibatch Loss= 0.3591, Training Accuracy= 0.880<br>
Step 1000, Minibatch Loss= 0.1403, Training Accuracy= 1.000<br>
Step 1100, Minibatch Loss= 0.0560, Training Accuracy= 1.000<br>
Step 1200, Minibatch Loss= 0.0284, Training Accuracy= 1.000<br>
Step 1300, Minibatch Loss= 0.0063, Training Accuracy= 1.000<br>
Step 1400, Minibatch Loss= 0.0109, Training Accuracy= 1.000<br>
Step 1500, Minibatch Loss= 0.0078, Training Accuracy= 1.000<br>
Optimization Finished!
Testing Accuracy: 0.4087102
<br>learning rate = 0.0010, img size = 64, dropout = 0.500<br>
3\*conv:64,128,256<br>
3\*fc:512,256,256

learning rate = 0.001; img size = 64; dropout = 0.5<br>
Step 1, Minibatch Loss= 3.2927, Training Accuracy= 0.060<br>
Step 100, Minibatch Loss= 2.7346, Training Accuracy= 0.040<br>
Step 200, Minibatch Loss= 2.3892, Training Accuracy= 0.180<br>
Step 300, Minibatch Loss= 2.2070, Training Accuracy= 0.280<br>
Step 400, Minibatch Loss= 1.9254, Training Accuracy= 0.400<br>
Step 500, Minibatch Loss= 1.1103, Training Accuracy= 0.660<br>
Step 600, Minibatch Loss= 0.7725, Training Accuracy= 0.740<br>
Step 700, Minibatch Loss= 1.0170, Training Accuracy= 0.620<br>
Step 800, Minibatch Loss= 0.2346, Training Accuracy= 0.960<br>
Step 900, Minibatch Loss= 0.1016, Training Accuracy= 0.980<br>
Step 1000, Minibatch Loss= 0.0475, Training Accuracy= 0.980<br>
Optimization Finished!
Testing Accuracy: 0.35008374

learning rate = 0.001000, img size = 64, dropout = 0.5<br>
Step 1, Minibatch Loss= 2.7135, Training Accuracy= 0.100<br>
Step 100, Minibatch Loss= 2.7063, Training Accuracy= 0.040<br>
Step 200, Minibatch Loss= 2.7133, Training Accuracy= 0.040<br>
Step 300, Minibatch Loss= 2.4252, Training Accuracy= 0.200<br>
Step 400, Minibatch Loss= 2.0151, Training Accuracy= 0.280<br>
Step 500, Minibatch Loss= 1.0233, Training Accuracy= 0.680<br>
Step 600, Minibatch Loss= 0.5331, Training Accuracy= 0.860<br>
Step 700, Minibatch Loss= 0.2126, Training Accuracy= 0.940<br>
Step 800, Minibatch Loss= 0.0660, Training Accuracy= 0.980<br>
Step 900, Minibatch Loss= 0.0105, Training Accuracy= 1.000<br>
Step 1000, Minibatch Loss= 0.0058, Training Accuracy= 1.000<br>
Optimization Finished!
Testing Accuracy: 0.33366835<br>
增大卷积层size


learning rate = 0.001; img size = 64; dropout = 0.875<br>
Step 1, Minibatch Loss= 2.8193, Training Accuracy= 0.100<br>
Step 100, Minibatch Loss= 2.7312, Training Accuracy= 0.060<br>
Step 200, Minibatch Loss= 2.6937, Training Accuracy= 0.020<br>
Step 300, Minibatch Loss= 2.2853, Training Accuracy= 0.220<br>
Step 400, Minibatch Loss= 2.0213, Training Accuracy= 0.320<br>
Step 500, Minibatch Loss= 1.7134, Training Accuracy= 0.420<br>
Step 600, Minibatch Loss= 0.4333, Training Accuracy= 0.960<br>
Step 700, Minibatch Loss= 0.4460, Training Accuracy= 0.880<br>
Step 800, Minibatch Loss= 0.1178, Training Accuracy= 1.000<br>
Step 900, Minibatch Loss= 0.1311, Training Accuracy= 0.980<br>
Step 1000, Minibatch Loss= 0.0548, Training Accuracy= 0.980<br>
Optimization Finished!
Testing Accuracy: 0.31289783

with learning rate = 0.001<br>
Step 1, Minibatch Loss= 1213625344.0000, Training Accuracy= 0.020<br>
Step 100, Minibatch Loss= 43687400.0000, Training Accuracy= 0.510<br>
Step 200, Minibatch Loss= 18261356.0000, Training Accuracy= 0.610<br>
Step 300, Minibatch Loss= 7593531.5000, Training Accuracy= 0.730<br>
Step 400, Minibatch Loss= 3516608.7500, Training Accuracy= 0.780<br>
Step 500, Minibatch Loss= 386.9200, Training Accuracy= 0.990<br>
Optimization Finished!
Testing Accuracy: 0.26599666

learning rate 0.001<br>
Step 1, Minibatch Loss= 11922490.0000, Training Accuracy= 0.050<br>
Step 100, Minibatch Loss= 1126417.1250, Training Accuracy= 0.150<br>
Step 200, Minibatch Loss= 901864.8750, Training Accuracy= 0.150<br>
Step 300, Minibatch Loss= 619173.6250, Training Accuracy= 0.100<br>
Step 400, Minibatch Loss= 469205.9062, Training Accuracy= 0.200<br>
Step 500, Minibatch Loss= 420558.8438, Training Accuracy= 0.100<br>
Step 600, Minibatch Loss= 220658.9688, Training Accuracy= 0.150<br>
Step 700, Minibatch Loss= 178520.2188, Training Accuracy= 0.150<br>
Step 800, Minibatch Loss= 146223.2812, Training Accuracy= 0.200<br>
Step 900, Minibatch Loss= 78263.0156, Training Accuracy= 0.100<br>
Step 1000, Minibatch Loss= 15069.7920, Training Accuracy= 0.150<br>
Step 1100, Minibatch Loss= 19763.5781, Training Accuracy= 0.150<br>
Step 1200, Minibatch Loss= 5730.0762, Training Accuracy= 0.100<br>
Step 1300, Minibatch Loss= 3.1026, Training Accuracy= 0.100<br>
Step 1400, Minibatch Loss= 1577.2947, Training Accuracy= 0.050<br>
Step 1500, Minibatch Loss= 3.5202, Training Accuracy= 0.000<br>
Step 1600, Minibatch Loss= 2.7354, Training Accuracy= 0.100<br>
Step 1700, Minibatch Loss= 3.0511, Training Accuracy= 0.050<br>
Step 1800, Minibatch Loss= 3.0807, Training Accuracy= 0.000<br>
Step 1900, Minibatch Loss= 2.8522, Training Accuracy= 0.000<br>
Step 2000, Minibatch Loss= 2.8605, Training Accuracy= 0.000<br>
Optimization Finished!
Testing Accuracy: 0.07638191